In [2]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms
import matplotlib.pyplot as plt
import torchvision

In [3]:
data_train = pd.read_csv('fashion-mnist_train.csv/fashion-mnist_train.csv')
data_test = pd.read_csv('fashion-mnist_test.csv/fashion-mnist_test.csv')

In [4]:
data_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
class MyDataset():
    def __init__(self, data, transform = None):
        self.fashion_mnist = list(data.values)
        self.transform = transform
        label, img = [],[]
        for one_line in self.fashion_mnist:
            label.append(one_line[0])
            img.append(one_line[1:])
        self.label = np.asarray(label)
        self.img = np.asarray(img).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, CHANNEL).astype('float32')

    def __getitem__(self, item):
        label, img = self.label[item], self.img[item]
        if self.transform is not None:
            img = self.transform(img)

        return label, img

    def __len__(self):
        return len(self.label)

In [6]:
BATCH_SIZE = 50
LR = 0.005
NUM_CLASS = 10
IMAGE_SIZE = 28
CHANNEL = 1
Train_epoch = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CLASS_CLOTHING = {0 :'T-shirt/top',
                  1 :'Trouser',
                  2 :'Pullover',
                  3 :'Dress',
                  4 :'Coat',
                  5 :'Sandal',
                  6 :'Shirt',
                  7 :'Sneaker',
                  8 :'Bag',
                  9 :'Ankle boot'}

In [7]:
My_transform = transforms.Compose([
    transforms.ToTensor(),# default : range [0, 255] -> [0.0,1.0]
])

Train_data = MyDataset(data_train, transform=My_transform)
Test_data = MyDataset(data_test, transform=My_transform)

Train_dataloader = DataLoader(dataset=Train_data,
                              batch_size = BATCH_SIZE,
                              shuffle=False
                              )
Test_dataloader = DataLoader(dataset=Test_data,
                              batch_size = BATCH_SIZE,
                              shuffle=False
                              )

## First Model

In [8]:
class My_Model2(nn.Module):
    def __init__(self, num_of_class):
        super(My_Model2, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(28, 16),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Linear(28, 32),
            nn.Softmax())
        self.layer3 = nn.Sequential(
            nn.Linear(32, 7*8),
            nn.Sigmoid())
        self.fc = nn.Linear(7 * 7 * 32, num_of_class)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(x)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [100]:
def train():
    model2 = My_Model2(NUM_CLASS).to(device)
    optimizer = torch.optim.SGD(model2.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(1, Train_epoch + 1):
        for batch_id, (label, image) in enumerate(Train_dataloader):
            label, image = label.to(device), image.to(device)
            output = model2(image)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_id % 1000 == 0:
                print('Loss :{:.4f} Epoch[{}/{}]'.format(loss.item(), epoch, Train_epoch))
    return model2

In [101]:
def test(model2):
    with torch.no_grad():
        correct = 0
        total = 0
        for label , image in Test_dataloader:
            image = image.to(device)
            label = label.to(device)
            outputs = model2(image)
            predicted = torch.argmax(outputs,dim=1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
        print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

In [102]:
if __name__ == '__main__':
    model2 = train()
    test(model2)

Loss :2.3815 Epoch[1/5]
Loss :2.3703 Epoch[1/5]
Loss :2.3448 Epoch[2/5]
Loss :2.3686 Epoch[2/5]
Loss :2.3439 Epoch[3/5]
Loss :2.3670 Epoch[3/5]
Loss :2.3431 Epoch[4/5]
Loss :2.3653 Epoch[4/5]
Loss :2.3423 Epoch[5/5]
Loss :2.3637 Epoch[5/5]
Test Accuracy of the model on the test images: 10.0 %


## First Model With regularization

In [9]:
def train():
    model2 = My_Model2(NUM_CLASS).to(device)
    optimizer = torch.optim.SGD(model2.parameters(), lr=LR,weight_decay=0.01)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(1, Train_epoch + 1):
        for batch_id, (label, image) in enumerate(Train_dataloader):
            label, image = label.to(device), image.to(device)
            output = model2(image)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_id % 1000 == 0:
                print('Loss :{:.4f} Epoch[{}/{}]'.format(loss.item(), epoch, Train_epoch))
    return model2

In [10]:
def test(model2):
    with torch.no_grad():
        correct = 0
        total = 0
        for label , image in Test_dataloader:
            image = image.to(device)
            label = label.to(device)
            outputs = model2(image)
            predicted = torch.argmax(outputs,dim=1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
        print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

In [11]:
if __name__ == '__main__':
    model2 = train()
    test(model2)

C:\Users\user\Anaconda3\lib\site-packages\torch\nn\modules\container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Loss :2.3575 Epoch[1/5]
Loss :2.3719 Epoch[1/5]
Loss :2.3455 Epoch[2/5]
Loss :2.3698 Epoch[2/5]
Loss :2.3445 Epoch[3/5]
Loss :2.3679 Epoch[3/5]
Loss :2.3435 Epoch[4/5]
Loss :2.3661 Epoch[4/5]
Loss :2.3426 Epoch[5/5]
Loss :2.3645 Epoch[5/5]
Test Accuracy of the model on the test images: 10.0 %


## Second Model

In [21]:
class My_Model(nn.Module):
    def __init__(self, num_of_class):
        super(My_Model, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7 * 7 * 32, num_of_class)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [18]:
def train():
    model = My_Model(NUM_CLASS).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = LR)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(1, Train_epoch + 1):
        for batch_id, (label, image) in enumerate(Train_dataloader):
            label, image = label.to(device), image.to(device)
            output = model(image)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_id % 1000 == 0:
                print('Loss :{:.4f} Epoch[{}/{}]'.format(loss.item(), epoch, Train_epoch))
    return model

In [19]:
def test(model):
    with torch.no_grad():
        correct = 0
        total = 0
        for label , image in Test_dataloader:
            image = image.to(device)
            label = label.to(device)
            outputs = model(image)
            predicted = torch.argmax(outputs,dim=1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
        print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

In [20]:
if __name__ == '__main__':
    model = train()
    test(model)

Loss :2.4226 Epoch[1/5]
Loss :0.3945 Epoch[1/5]
Loss :0.4498 Epoch[2/5]
Loss :0.3525 Epoch[2/5]
Loss :0.3775 Epoch[3/5]
Loss :0.3162 Epoch[3/5]
Loss :0.2878 Epoch[4/5]
Loss :0.2691 Epoch[4/5]
Loss :0.2358 Epoch[5/5]
Loss :0.2405 Epoch[5/5]
Test Accuracy of the model on the test images: 90.01 %


## Second Model With regularization

In [22]:
def train():
    model = My_Model(NUM_CLASS).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = LR,weight_decay=0.01)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(1, Train_epoch + 1):
        for batch_id, (label, image) in enumerate(Train_dataloader):
            label, image = label.to(device), image.to(device)
            output = model(image)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_id % 1000 == 0:
                print('Loss :{:.4f} Epoch[{}/{}]'.format(loss.item(), epoch, Train_epoch))
    return model

In [23]:
def test(model):
    with torch.no_grad():
        correct = 0
        total = 0
        for label , image in Test_dataloader:
            image = image.to(device)
            label = label.to(device)
            outputs = model(image)
            predicted = torch.argmax(outputs,dim=1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
        print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

In [ ]:
if __name__ == '__main__':
    model = train()
    test(model)

Loss :2.6361 Epoch[1/5]
Loss :0.5261 Epoch[1/5]
Loss :0.5501 Epoch[2/5]
Loss :0.5253 Epoch[2/5]
Loss :0.4838 Epoch[3/5]
Loss :0.5015 Epoch[3/5]
Loss :0.4494 Epoch[4/5]
